The following Zi failed
```
儿 猴 龙 头 死 毛 皮 眉 耳 肉 一 午 四 寅 甲 癸 零 丿 彡 攵 氵犭
```

In [1]:
from time import time
from datetime import datetime
import re
from pathlib import Path

import pandas as pd 

import ollama

from utils import *

In [2]:
model_name = "qwen2.5"  # "llama3.1"
pattern = r'```json\s*(.*?)\s*```'
file_log = "char360-qwen2.5.log"

In [3]:
prompt_zi = """
You are an expert in Chinese language, 

can you generate a holistic view on this chinese character  {zi}
in terms of the following attributes:
含义
字形
读音 
字源
常用词组
成语
例句
短故事
诗词
图片
音频 
视频 
电影
参考资料
有趣网站

(1) give the answer in Chinese 
(2) format the answer in valid json and ensure quotes are properly escaped (specifically avoid double-quotes nested in doube-quotes)
(3) whenever possible, give 5 or more examples for the following attributes:

常用词组
成语
例句
短故事
诗词
图片
音频 
视频 
电影
参考资料
有趣网站

"""

In [4]:
df = pd.read_csv("char360_fib_num.csv")
zi_fib_nums = df.to_dict("records")

In [5]:
for xx in zi_fib_nums:
    zi, fib_num = xx['zi'], xx['fib_num']
    fib_num_str = str(fib_num).zfill(4)
    fp = Path(f"./llm_models/{model_name}") / f"{fib_num_str}-{zi}-1.json"    
    if fp.exists():
        # log_msg(f"SKIP {zi}: Already processed - {fp} ", file_log)
        continue

    print(f"Processing {zi} ...")
    
    ts_now = datetime.now()   
    ts_1 = time()
    try:
        resp = ollama.generate(model=model_name, prompt=prompt_zi.format(zi=zi))
    except Exception as e:
        log_msg(f"[ERROR] {zi} : Failed to call ollama.generate()", file_log)
        continue
        
    ts_2 = time()
    del_t = ts_2 - ts_1
    del_t_str = f"{del_t:.3f}"
    json_raw = resp.get('response', "")
    if not json_raw: 
        log_msg(f"[ERROR] No LLM response: {zi}", file_log)
        continue

    log_msg(f"[{str(ts_now)}] Completed {zi} in {del_t_str} sec")

    match = re.search(pattern, json_raw, re.DOTALL)

    if match:
        json_string = match.group(1)
        if not json_string:
            log_msg(f"[WARN] No JSON extracted: {zi}")
            continue
            
        with open(fp, "w", encoding="utf-8") as f:
            f.write(json_string)
    else:
        log_msg(f"[ERROR] No JSON extracted: {zi}", file_log)

Processing 它 ...
[2024-09-20 18:26:34.590533] Completed 它 in 111.412 sec
Processing 对 ...
[2024-09-20 18:28:26.004111] Completed 对 in 159.693 sec
Processing 少 ...
[2024-09-20 18:31:05.699130] Completed 少 in 158.854 sec
Processing 川 ...
[2024-09-20 18:33:44.553678] Completed 川 in 100.468 sec
Processing 左 ...
[2024-09-20 18:35:25.022948] Completed 左 in 123.203 sec
Processing 己 ...
[2024-09-20 18:37:28.226443] Completed 己 in 79.711 sec
Processing 币 ...
[2024-09-20 18:38:47.939004] Completed 币 in 128.828 sec
Processing 市 ...
[2024-09-20 18:40:56.768564] Completed 市 in 121.384 sec
Processing 平 ...
[2024-09-20 18:42:58.153126] Completed 平 in 113.857 sec
Processing 年 ...
[2024-09-20 18:44:52.011175] Completed 年 in 152.357 sec
Processing 幺 ...
[2024-09-20 18:47:24.368329] Completed 幺 in 90.093 sec
Processing 应 ...
[2024-09-20 18:48:54.462964] Completed 应 in 73.626 sec
Processing 弟 ...
[2024-09-20 18:50:08.090429] Completed 弟 in 111.267 sec
Processing 当 ...
[2024-09-20 18:51:59.358212] Complete

KeyboardInterrupt: 